In [1]:
from dotenv import load_dotenv 
from openai import OpenAI
from testUtils import print_properties, save_pickle,load_pickle,num_tokens_from_messages
load_dotenv()
client = OpenAI()
selectedGPTmodel = "gpt-4o-mini"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Which city is China's capital?"}
]

## text generation/chat
https://platform.openai.com/docs/guides/text-generation/quickstart

```python
completion = client.chat.completions.create(
    model=selectedGPTmodel,
    messages= messages
)

save_pickle(completion,'completion-test1.pkl')
```

In [2]:
completion = load_pickle('completion-test1.pkl')
print(completion.choices[0].message)

ChatCompletionMessage(content='The capital of China is Beijing.', refusal=None, role='assistant', function_call=None, tool_calls=None)


### Returned object

- Type of `ChatCompletion`
- https://platform.openai.com/docs/api-reference/chat/object

In [3]:
type(completion)

openai.types.chat.chat_completion.ChatCompletion

In [4]:
print_properties(completion)

id:
  chatcmpl-AJDBs7ryJnfpfJDLg6xg2F9FFn0RD
choices:
  [0]:
    finish_reason:
      stop
    index:
      0
    logprobs:
      None
    message:
      content:
        The capital of China is Beijing.
      refusal:
        None
      role:
        assistant
      function_call:
        None
      tool_calls:
        None
created:
  1729142300
model:
  gpt-4o-mini-2024-07-18
object:
  chat.completion
service_tier:
  None
system_fingerprint:
  fp_e2bde53e6e
usage:
  completion_tokens:
    7
  prompt_tokens:
    23
  total_tokens:
    30
  completion_tokens_details:
    audio_tokens:
      None
    reasoning_tokens:
      0
  prompt_tokens_details:
    audio_tokens:
      None
    cached_tokens:
      0
_request_id:
  req_ed3d3ebdbc6c119a6eb99185f09b34fd


In [5]:
import json
completion_dict = completion.to_dict()
print(json.dumps(completion_dict, indent=2))

{
  "id": "chatcmpl-AJDBs7ryJnfpfJDLg6xg2F9FFn0RD",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The capital of China is Beijing.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1729142300,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_e2bde53e6e",
  "usage": {
    "completion_tokens": 7,
    "prompt_tokens": 23,
    "total_tokens": 30,
    "completion_tokens_details": {
      "reasoning_tokens": 0
    },
    "prompt_tokens_details": {
      "cached_tokens": 0
    }
  }
}


### Manage output tokens and Context

- https://platform.openai.com/docs/guides/text-generation/managing-context-for-text-generation
- https://cookbook.openai.com/examples/how_to_count_tokens_with_tiktoken

In [6]:
import tiktoken
tokenizer = tiktoken.encoding_for_model(selectedGPTmodel)
tokenizer.encode("tiktoken is great!"), \
[tokenizer.decode_single_token_bytes(token) for token in [83, 8251, 2488, 382, 2212, 0]]


([83, 8251, 2488, 382, 2212, 0],
 [b't', b'ikt', b'oken', b' is', b' great', b'!'])

In [7]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Which city is China's capital?"}
]
num_tokens_from_messages(messages, model=selectedGPTmodel)

23

**Remove the oldest user/assistant messages until within the limit**

In [8]:
# Function to manage context and limit tokens
def generate_response(model, messages,tokenizer, max_output_tokens=100, max_context_tokens=3000):
    total_tokens = num_tokens_from_messages(messages=messages, model=selectedGPTmodel)

    # Truncate context if it exceeds max_context_tokens
    while total_tokens > max_context_tokens:
        # Remove the oldest user/assistant messages until within the limit
        messages.pop(1)  # Popping after system message
        total_tokens = num_tokens_from_messages(messages,selectedGPTmodel)

    # Call the OpenAI API with controlled max_tokens
    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=max_output_tokens
    )

    return completion



# completion = generate_response(selectedGPTmodel, messages, 
#                                tokenizer=tokenizer,
#                                max_output_tokens=100, max_context_tokens=3000
#                                )

# save_pickle(completion,'completion-test2.pkl')

In [9]:
completion = load_pickle('completion-test2.pkl')
print(completion.choices[0].message)

ChatCompletionMessage(content="China's capital city is Beijing.", refusal=None, role='assistant', function_call=None, tool_calls=None)


In [10]:
print(completion.choices[0].message.content)

China's capital city is Beijing.


### Sturctured output

In [11]:
from pydantic import BaseModel

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

```python
completion = client.beta.chat.completions.parse(
    model= selectedGPTmodel,
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
    ],
    response_format=CalendarEvent,
)
completion_dict = completion.dict()  # Converts to a serializable dictionary
save_pickle(completion_dict,'completion-test3.pkl')
event = completion.choices[0].message.parsed # CalendarEvent
event

# CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob'])
```

In [12]:
completion_dict = load_pickle('completion-test3.pkl')
event = completion_dict["choices"][0]["message"]["parsed"]
event

{'name': 'Science Fair', 'date': 'Friday', 'participants': ['Alice', 'Bob']}

In [13]:
completion_dict

{'id': 'chatcmpl-AJN7XdVau6rPuQY5iLvpCBJ1ddNYx',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': '{"name":"Science Fair","date":"Friday","participants":["Alice","Bob"]}',
    'refusal': None,
    'role': 'assistant',
    'function_call': None,
    'tool_calls': [],
    'parsed': {'name': 'Science Fair',
     'date': 'Friday',
     'participants': ['Alice', 'Bob']}}}],
 'created': 1729180471,
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': 'fp_e2bde53e6e',
 'usage': {'completion_tokens': 17,
  'prompt_tokens': 92,
  'total_tokens': 109,
  'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}}